<a href="https://colab.research.google.com/github/SelinaX23/automatic_cat_feeder/blob/main/deep_learning_cat_face.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Preprocess Video


In [ ]:
#video_path = "/content/drive/MyDrive/cat/video/"
pic_path = "/content/drive/MyDrive/cat/camera_input/"
output_image_folder = "/content/drive/MyDrive/cat/to_label/"
LABELS = {val: id for id, val in enumerate(["persephone", "eris", "other"])}
labeled_folder = "/content/drive/MyDrive/cat/labeled/"

##  Parse images from video


In [ ]:
!pip install opencv-python tqdm
# Make folder to store the images
!mkdir -p images

In [ ]:
import cv2
import os
from tqdm.notebook import tqdm


def resize(input_path, output_folder, output_prefix, output_size=400):

    image = cv2.imread(input_path)
    if image is not None:
      w, h = image.shape[:2]
      size = min(w, h) // 2
      image = image[w//2-size: w//2 + size, h//2-size:h//2+size]

      output_path = os.path.join(output_folder, output_prefix)
      # shrink to 300 * 300 (we don't really need large picture)
      image = cv2.resize(image, (output_size, output_size))
      #image = cv2.flip(image, 0)

      cv2.imwrite(output_path, image)
    if os.path.exists(input_path):
      os.remove(input_path)



def resize_pic(input_folder, output_folder, output_size=400, start=0):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    files = [
        os.path.join(root, file_name)
        for root, _, files in os.walk(input_folder)
        for file_name in files
        if file_name.endswith(".jpg") or file_name.endswith(".jpeg")


    ]

    for input_path in tqdm(files):
      output_prefix = input_path.replace(input_folder, "")

      resize(
            input_path,
            output_folder,
            output_prefix,
            output_size=output_size

        )


resize_pic(pic_path, output_folder=output_image_folder)

# Label session



In [ ]:
!pip install ipywidgets

In [ ]:
import os
import functools

import ipywidgets as widgets
from IPython.core.display import Image, display, clear_output

def on_label(_, label, path):
    print(label, path)
    target = os.path.join(labeled_folder, label)
    if not os.path.exists(target):
        os.makedirs(target)
    base_name = os.path.basename(path)
    os.rename(path, os.path.join(target, base_name))
    clear_output()
    show_next()


def show_example(current_path):
    buttons = []
    for label in LABELS:
        button = widgets.Button(description=label)
        button.on_click(functools.partial(on_label, label=label, path=current_path))
        buttons.append(button)

    display(Image(filename=current_path))
    display(current_path)
    display(widgets.HBox(buttons))


def get_label_result():
    labeled_count = {
        label: len(os.listdir(os.path.join(labeled_folder, label)))
        for label in LABELS
        if os.path.exists(os.path.join(labeled_folder, label))
    }
    return sorted(labeled_count.items(), key=lambda x: -x[1])


def show_next():
    files = os.listdir(output_image_folder)
    print(len(files))
    print(get_label_result())
    if files:
        current_path = os.path.join(output_image_folder, files[0])
        show_example(current_path)
    else:
        print("All labeled")

show_next()

# Modeling


## Prepare Data

In [ ]:
import os

import numpy as np
import tensorflow as tf
import pathlib

LABELS = {val: id for id, val in enumerate(["persephone", "eris", "other"])}

labeled_folder = "/content/drive/MyDrive/cat/labeled/"
data_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255, validation_split=0.2)
train = data_gen.flow_from_directory(
    labeled_folder,
    target_size=(400, 400),
    class_mode='categorical',
    batch_size=32,
    subset="training"
)
validation = data_gen.flow_from_directory(
    labeled_folder,
    target_size=(400, 400),
    class_mode='categorical',
    batch_size=32,
    subset="validation"
)

## Build Model

In [ ]:
import random

from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import GlobalAveragePooling2D

import numpy as np

base_model = Xception(
    input_shape=(400, 400, 3),
    weights='imagenet', include_top=False)  # do not include the Fully connected layer
# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer -- let's say we have  classes
predictions = Dense(3, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=["accuracy"])

model.summary()

In [ ]:
# train the model on the new data for a few epochs
history = model.fit(
    train,
    epochs=20,
    steps_per_epoch=train.samples / train.batch_size,
    validation_data=validation
)

In [ ]:
import matplotlib.pyplot as plt
plt.style.use('dark_background')

# Get training and test loss histories
training_loss = history.history['loss']
test_loss = history.history['val_loss']

# Create count of the number of epochs
epoch_count = range(1, len(training_loss) + 1)

# Visualize loss history
plt.plot(epoch_count, training_loss, 'r--')
plt.plot(epoch_count, test_loss, 'b-')
plt.legend(['Training Loss', 'Test Loss'])
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show();

# Prediction


# New section

In [ ]:
from IPython.display import Image
from keras import utils
from keras.preprocessing import image

print(train.class_indices, validation.class_indices)
predict_to_cls = {v: k for k, v in validation.class_indices.items()}
def test(size=10):

    for img_path in random.sample(validation.filepaths, size):
        display(Image(filename=img_path))
        img = utils.load_img(img_path, target_size=(400, 400))
        x = utils.img_to_array(img)
        x = np.expand_dims(x, axis=0)

        images = np.vstack([x]) / 255.
        predict = model.predict(images)
        print(predict, np.argmax(predict[0]))
        str_class = predict_to_cls[np.argmax(predict[0])]
        print("label:", img_path.rsplit("/", 2)[-2])
        print('Predicted:', str_class)


test(10)

In [ ]:
from IPython.display import Image
from keras import utils
from keras.preprocessing import image
import os
from tqdm.notebook import tqdm

test = "/content/drive/MyDrive/cat/test-camera/"

predict_to_cls = {v: k for k, v in validation.class_indices.items()}

files = [
    os.path.join(root, file_name)
    for root, _, files in os.walk(test)
    for file_name in files
    ]


for img in tqdm(files):
    img = utils.load_img(img, target_size=(400, 400))
    display(img)

    x = utils.img_to_array(img)
    x = np.expand_dims(x, axis=0)

    images = np.vstack([x]) / 255.
    predict = model.predict(images)
    str_class = predict_to_cls[np.argmax(predict[0])]
    print('Predicted:', str_class)


In [ ]:
import tensorflow as tf
export_dir = '/content/drive/My Drive/projects/automatic_feeder/saved_model/model2'
tf.saved_model.save(model, export_dir)
converter = tf.lite.TFLiteConverter.from_saved_model(export_dir)
tflite_model = converter.convert()
tflite_model_file = pathlib.Path('model2.tflite')
tflite_model_file.write_bytes(tflite_model)

try:
  from google.colab import files
  files.download(tflite_model_file)
except:
  pass